# RBF网络模型

In [25]:
import torch
import torch.nn as nn

import numpy as np

class RBFKernelStd(nn.Module):
    def __init__(self):
        super(RBFKernelStd, self).__init__()
        self.coefficient = 1 / np.sqrt(2 * np.pi)
        
    def forward(self, x):
        return torch.exp(-0.5 * x * x) * self.coefficient

class RBF1D(nn.Module):
    def __init__(self, n_params=20):
        super(RBF1D, self).__init__()
        self.n_params = n_params
        self.kernel = RBFKernelStd()
        self.one = torch.Tensor([1.])
        self.a = nn.Parameter(torch.ones(self.n_params), requires_grad=True)
        self.b = nn.Parameter(torch.ones(self.n_params), requires_grad=True)
        self.linear = nn.Linear(n_params, 1, bias=True)
        #self.w = nn.Parameter(torch.ones(self.n_params + 1), requires_grad=True)
        self.init()
    
    def init(self):
        self.a.data.normal_(0, 0.2)
        self.b.data.normal_(0, 0.2)
        #self.w.data.normal_(0, 0.2)
        self.linear.weight.data.normal_(0, 0.2)
        
    def forward(self, x):
        g = self.kernel(self.a * x + self.b)
        #g_aug = torch.cat([g, self.one], dim=0)
        y = self.linear(g)#torch.dot(self.w, g_aug)
        return y

# RBF拟合流程

In [ ]:
import torch.optim as optim

from matplotlib import pyplot as plt


class GUI_Config:
    function_range = [-1, 1]#[-400, 400]
    canvas_size = [600, 600]
    background_color = 0x112F41
    line_segments = []
    points = []
    
    @staticmethod
    def normalized(data):
        a, b = GUI_Config.function_range[0], GUI_Config.function_range[1]
        
        return tuple((d - a) / (b - a) for d in data)
    
    @staticmethod
    def denormalized(data):
        a, b = GUI_Config.function_range[0], GUI_Config.function_range[1]
        return tuple(a * (1 - d) + b * d for d in data)
    
    @staticmethod
    def add_line_segments(rbf_training):
        GUI_Config.line_segments = []
        for x in range(0, GUI_Config.canvas_size[0] + 1, 1):
            xn = x / GUI_Config.canvas_size[0]
            xd = GUI_Config.denormalized((xn,))[0]
            yd = rbf_training.evaluate(xd)
            yn = GUI_Config.normalized((yd,))[0]
            GUI_Config.line_segments.append((xn, yn))
            
    
    @staticmethod
    def draw():
        if len(GUI_Config.line_segments) >= 2:
            GUI_Config.line_segments.sort(key=lambda x: x[0])
            for i in range(0, len(GUI_Config.line_segments) - 1):
                plt.plot(GUI_Config.line_segments[i], GUI_Config.line_segments[i + 1])
        # for p in GUI_Config.points:
            # gui.circle(p, 0xFF0000, 8)
    

class RBF_Training:
    def __init__(self, **kwargs):
        self.span = kwargs.get('span', 8)
        self.lr = kwargs.get('lr', 0.05)
        self.max_epoch = kwargs.get('max_epoch', 1000)
        self.epsilon = kwargs.get('epsilon', 1e-6)
        self.model = RBF1D(self.span)
        self.datas = {}
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
        self.loss_fun = nn.MSELoss()
        self.last_loss = np.nan
        
    def update_model(self, **kwargs):
        self.span = kwargs.get('span', self.span)
        self.lr = kwargs.get('lr', self.lr)
        self.max_epoch = kwargs.get('max_epoch', self.max_epoch)
        self.epsilon = kwargs.get('epsilon', self.epsilon)
        self.model = RBF1D(self.span)
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
        
    def train(self):
        self.model.init()
        self.model.train()
        for epoch in range(self.max_epoch):
            for x in self.datas:
                y = self.datas[x]
                xt = torch.Tensor([x])
                yt = torch.Tensor([y])
                
                self.optimizer.zero_grad()
                y_prediction = self.model(xt)
                loss = self.loss_fun(y_prediction, yt)
                loss.backward()
                self.optimizer.step()

            print("[Epoch: {:>4}] loss = {:>.9}".format(epoch + 1, loss.item()))
            if loss.item() < self.epsilon:
                break
        self.last_loss = loss.item()
        print(" [*] Training finished!")
    
    def evaluate(self, x):
        y = self.model(torch.Tensor([x])).item()
        return y

# 设置训练参数 + 导入数据

### 模拟交互后识别到的6个坐标

In [15]:
collection=((1,2),(3,4),(1.3,5.1),(4.4,2.2),(0.1,7.9),(9,9)) 

### 模拟交互后设置的训练参数

In [27]:
lr_slide = 1 # gui.slider('learning rate', 1e-3, 1, step=1e-3)
# lr_slide.value = rbf_training.lr
epsilon_slide = 1 # gui.slider('epsilon * %d'%esilon_times, 1e-3, 1, step=1e-3)
# epsilon_slide.value = rbf_training.epsilon * esilon_times
max_epoch_slide = 100 # gui.slider('max epoch', 100, 5000, step=100)
# max_epoch_slide.value = rbf_training.max_epoch
span_slide = 10 # gui.slider('span', 5, 50, step=1)
# span_slide.value = rbf_training.span

# 导入输入训练模型，输出结果
使用matplotlib的pyplot呈现拟合函数

In [ ]:
if __name__ == "__main__":
    esilon_times = 10000
    rbf_training = RBF_Training()
    # gui = GUI("RBF Network Fitting (right click to add point)", (GUI_Config.canvas_size[0], GUI_Config.canvas_size[1]), background_color=GUI_Config.background_color)
    # plot_x=[0:0.01:GUI_Config.canvas_size[0]]
    # plot_y=[0:0.01:GUI_Config.canvas_size[1]]
    # KEY_CLEAR = gui.button('clear')
    # KEY_TRAIN = gui.button('train')
    # lr_slide = gui.slider('learning rate', 1e-3, 1, step=1e-3)
    # lr_slide.value = rbf_training.lr
    # epsilon_slide = gui.slider('epsilon * %d'%esilon_times, 1e-3, 1, step=1e-3)
    # epsilon_slide.value = rbf_training.epsilon * esilon_times
    # max_epoch_slide = gui.slider('max epoch', 100, 5000, step=100)
    # max_epoch_slide.value = rbf_training.max_epoch
    # span_slide = gui.slider('span', 5, 50, step=1)
    # span_slide.value = rbf_training.span
 
    # while gui.running:
    #     event = None
    #     while gui.get_event():
    #         event = gui.event
    #         print(event.pos, event.key, event.type)
    #         break
    #     max_epoch_slide.value = int(max_epoch_slide.value)
    #     span_slide.value = int(span_slide.value)

    rbf_training.update_model(lr=lr_slide, epsilon=epsilon_slide / esilon_times,max_epoch=int(max_epoch_slide),span=int(span_slide))

        # if event:
            # print(event.pos, event.key, event.type)
            # if event.key == GUI.RMB and event.type == GUI.RELEASE:
            #     print('Add point')
            #     mouse_x, mouse_y = gui.get_cursor_pos()
            #     GUI_Config.points.append((mouse_x, mouse_y))
            #     canvas_x, canvas_y = GUI_Config.denormalized((mouse_x, mouse_y))
            #     rbf_training.datas[canvas_x] = canvas_y
    
    print('Add point')
    for i in collection:
      GUI_Config.points.append(i)
      canvas_x, canvas_y = GUI_Config.denormalized(i)
      rbf_training.datas[canvas_x] = canvas_y
            
            # if event.key == KEY_CLEAR or event.key == GUI.SPACE and event.type == GUI.RELEASE:
                # print('Clear')
                # rbf_training.datas = {}
                # GUI_Config.points = []
                # GUI_Config.line_segments = []
                
            # if event.key == KEY_TRAIN:
    print('Train')
    if len(rbf_training.datas) >= 2:
      GUI_Config.line_segments = []
            # GUI_Config.draw(gui)
      GUI_Config.draw()
      print('Last loss : ', rbf_training.last_loss) # gui.text('Last loss : %.9f'%rbf_training.last_loss, (0.01, 0.99), 24)
            # gui.show()
      rbf_training.train()
      GUI_Config.add_line_segments(rbf_training)
                    
    GUI_Config.draw()
    print('Last loss : ',rbf_training.last_loss) # gui.text('Last loss : %.9f'%rbf_training.last_loss, (0.01, 0.99), 24)
    
        # gui.show()
        # event = None
    # gui.close()